# Imports

In [1]:
import numpy as np
import pandas as pd
import random

# Loading the Data and Discarding What is Not Needed

In [2]:
data = pd.read_csv('./clean_data.csv',sep='\t',encoding='utf-8',index_col=0)
data.drop(labels=['Date','HomeTeam','AwayTeam',
                  'FTHG','FTAG','HS','AS','HST',
                  'AST','HTHG','HTAG','HF','AF',
                  'FTR','HTR','HY','AY','HR',
                  'AR','Season','WHH','WHD','WHA'],
         inplace=True,
         axis=1)
# Redorder the columns so we have home statistics, then away statistics
# then betting odds and finally the classifier.
data = data[['HGS','HGA','HYC','HRC','HWW','AGS','AGA','AYC','ARC','AWW','watch']]

We insert a column of ones at the beginning of the DataFrame to allow the regression to fit with an constant term, also know as the intercept.

In [3]:
data['intercept'] = np.ones((data.shape[0], 1))

In [4]:
data.shape

(6246, 12)

# Defining Variables and the Problem at Hand

There are going to be a lot of variables to keep track of so let's carefully define them and explain what they are in the context of the data above.

Let the matrix $X$, represent the data consisting of $m$ observations of $d$ continuous predictors. In our case, $X$ is the dataframe above **without** the 'watch' column. For us, $X$ will be a $6246 \times 11$ matrix, remembering we have added a column of ones to allow an intercept to be fitted.

Let $y$ represent the column vector of labels with $m$ rows. In our case, this is the 'watch' column of the dataframe with $6246$ entries.

Let $\theta$ also be a column vector, with $d$  entries. In our case $d = 11$.

A hyperplane is then defined by $\theta^Tx = 0$, where $x^T = (1, x_1, x_2, ... , x_d)$ is a general point in $d$-space, which splits the space into two parts. We can then classify points based on what side of the plane they are on. The 2d example of this is $\theta^T = (a, b, c)$ and $x^T = (1, x, y)$ where $\theta^Tx = a + bx + by = 0$ defines a line splitting the plane into 2 parts.

The problem at hand is to find values for the $\theta$ vector which split the space into two parts to match our binary classifier.

This is called a linear discriminant because we split the space using a plane with no curvature. This is the reason that the regression isn't going to perform particularly well. There is no reason to assume that the football data can be split by a non-curved boundary hyperplane.

An optimal solution for $\theta$ always exists to maximize correct prediction but it is computationally expensive to compute. Rather than finding the best values for $\theta$ we incrementally improve an initial guess moving closer to the optimal solution using a technqiue called gradient descent.

Imagine an egg cup where the optimal solution is the lowest point in the bowl. We start by randomly choosing a point on the bowl. By finding the gradient at that point we know the direction needed to travel towards the bottom. We move our position by some small amount (referred to as ALPHA in this notebook) towards the bottom of the egg cup. Then we recalculate the gradient and repeat. Does the name gradent descent make sense now?

# Implementing Logistic Regression

A [sigmoid function](https://www.wikiwand.com/en/Sigmoid_function) is needed to ensure values between 0 and 1 are generated to represent probability. We use the logistic sigmoid function below.

$$S(Y) = \frac{1}{1 + e^{-Y}}$$

We define it now to be used later.

In [5]:
# Applies the sigmoid function elementwise
def sigmoid(Y):
    return 1.0 / (1.0 + np.exp(-Y))

A logisitic regression is fit by *maximum likelihood* which means maximizing the probability that the correct labels are found given the training data and regression coefficients. This means we are maximizing $P(y|X,\theta)$.

For the regression to be fit, we need to introduce a penalty for misclassification that can then be minimized. This is called a cost function which we refer to as $C$. Here is the cost for a certain vector $\theta$ missclassifying:

$C(\theta) = -\log(S(X\theta))$ if $y = 1$

$C(\theta) = -\log(1 - S(X\theta))$ if $y = 0$

This cost functions punishes predictions close to 0 for rows with a prediction of 1 and vice versa. For example, if $S(X\theta)$ is close to zero meaning the logistic regression predicts a classification of $0$ but actualy $y=1$ then $-\log(S(X\theta))$ is going to be a large value which tends to infinity as the predcition gets closer to zero (ie worse and worse).

By assuming that all the joint observations of predictor and labels are independent and identically distributed (i.i.d), this definition leads to the log-likelihood equation becoming

$
\begin{eqnarray}
 P(y|X,\theta) & = & P(y_0, y_1, \cdots , y_{m-1}|x_0, x_1, \cdots , x_{m-1}, \theta) \\
 & = & P(y_0|x_0,\theta) P(y_1|x_1,\theta) \cdots P(y_{m-1}|x_{m-1},\theta) \\
 & = & \prod_{i=0}^{m-1}P(y_i|x_i,\theta) \\
\end{eqnarray}
$

and as we use the *log*-likelihood, this results in

$
\begin{eqnarray}
 L(\theta ; y, X) & = & \log \left\{ \prod_{i=0}^{m-1}P(y_i|x_i,\theta) \right\} \\
 & = & \sum_{i=0}^{m-1} \log (P(y_i|x_i,\theta))
\end{eqnarray}
$

Under the assumption that $S$ is a good function to model probability and the i.i.d assumption above, the log-likelihood equation can be equivalently written in vector notation to avoid the use of a summation as

$$L(\theta) = y^TX\theta + u^T\ln S(-X\theta)$$

where $u$ is a row vector of $m$ ones.

This function is defined below:

In [6]:
# Applies log-likelihood function to theta, given 
# y (classes) and X (input data)
def log_likelihood(theta, y, X):
    u = np.ones((len(X), 1))
    z = X @ theta
    return y.T @ z + u.T @ (np.log(sigmoid(-z)))

The gradient of the log likelihood equation is needed to perform gradient descent. Due to the fact that the cost function is convex, gradient descent is guaranteed to find the global minimum eventually. But it may take far too long in practice so we iterate for as long as we can in the situation. Here is the gradient:

$$\nabla_\theta L(\theta) = X^T [ y - G(X\theta) ]$$

In [7]:
def grad_log_likelihood(theta, y, X):
    error = y - sigmoid(X @ theta)
    return X.T.dot(error)

Now we are reading to perform gradient descent. We allow the setting of parameters for ALPHA the amount we move in each iteration, often called the learning rate and MAX_STEPS the number of times we run the iteration. This function outputs an approximation of theta.

In [8]:
def logistic_regression(X, y, ALPHA=0.1, MAX_STEPS=250):
    y = y.astype(dtype=float)
    
    # Initialize theta as zeros
    theta = np.zeros(X.shape[1])

    # Using gradient descent incrementally improve
    # the theta values towards the optimum solution
    for t in range(MAX_STEPS):
        delta_t = grad_log_likelihood(theta, y, X)
        delta_t = delta_t / np.linalg.norm(delta_t, ord=2)
        theta = theta + ALPHA * delta_t        
    return theta

# Fitting the Regression

First we split the data into training and testing. We will fit to the training data and then evaluate the accuracy by predicting the test data.

In [9]:
random.seed(314) # Pi
train_indices = random.sample(range(len(data)), int(0.8 * len(data)))
test_indices = [i for i in range(len(data)) if i not in train_indices]
train = data.iloc[train_indices]
test = data.iloc[test_indices]
print(train.shape)
print(test.shape)

(4996, 12)
(1250, 12)


We run the logistic regression function on the predictor columns and the result column. I have set MAX_STEPS as high as I am willing to wait for.

In [10]:
theta = logistic_regression(train.drop('watch', axis=1),
                            train['watch'],
                            ALPHA=0.01,
                            MAX_STEPS=1000000)

Here are coefficients of the regression equation. Note that the variables which seem to have the greatest influence on the prediciton are HWW and AWW. This makes sense as these are the proportion of past games in the season that have been worth watching!

In [11]:
print(theta)

HGS          0.050578
HGA         -0.061416
HYC          0.044441
HRC          0.146679
HWW          0.506910
AGS          0.014740
AGA         -0.067408
AYC         -0.037042
ARC          0.097068
AWW          0.443800
intercept   -0.933445
dtype: float64


We calculate the probability of each game being worth watching by multiplying the test data by the regression coefficients and applying the sigmoid function to restrict the values between 0 and 1. Unfortunately, the probabilities seems to be almost all between 0.2 and 0.4. It doesn't look like there are many games at all that the logistic regression was confident of being exciting. In fact it is normal to classify rows with a probability over 0.5 as being 1 and 0 otherwise. Well for this data, we would only predict one game as being probably worth watching!

In [12]:
scores = test.drop('watch', axis=1) @ theta
predictions = sigmoid(scores)

print(predictions.sample(20, random_state=101))

4599    0.323108
5792    0.331601
7       0.441372
5701    0.344439
145     0.343601
2361    0.332289
4096    0.430158
3289    0.332799
639     0.324616
6115    0.371568
4612    0.313670
1263    0.346605
4044    0.376662
518     0.340668
3647    0.331257
5116    0.334173
573     0.338910
3546    0.356114
1877    0.325884
734     0.357390
dtype: float64


To check the correctness of my implementation I will use the LogisticRegression function from Scikit-Learn

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [14]:
# We use the 'saga' solver which the documentation recommends for large
# datasets and we set the max iterations to be the same as above.
clf = LogisticRegression(solver='saga',max_iter=10000000, fit_intercept=False, random_state=101)
clf.fit(train.drop('watch', axis=1), train['watch'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=101, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

Very close (though not equal) regression coefficients are found which shows that my implementation is working.

In [15]:
for z in zip(np.round((clf.coef_[0] + clf.intercept_), decimals=8), 
             list(np.round(theta, decimals=8))):
    print(f'SK: {z[0]}, Me: {z[1]}')

SK: 0.04719766, Me: 0.05057817
SK: -0.06386703, Me: -0.06141576
SK: 0.03659465, Me: 0.04444093
SK: 0.133882, Me: 0.14667883
SK: 0.48749553, Me: 0.50691018
SK: 0.01030128, Me: 0.01473978
SK: -0.07179634, Me: -0.06740772
SK: -0.04433594, Me: -0.03704187
SK: 0.08878757, Me: 0.09706804
SK: 0.43073881, Me: 0.4438
SK: -0.89385275, Me: -0.93344532


In [16]:
probs = clf.predict_proba(test.drop('watch', axis=1))
probs = [prob[1] for prob in probs]

In [17]:
preds = clf.predict(test.drop('watch', axis=1))
results = pd.DataFrame(data={'probs': probs, 'original': test['watch']})
results['prediction'] = results['probs'].apply(lambda x: 1 if x > 0.5 else 0)
print((results['original'] == results['prediction']).sum())
confusion_matrix(results['prediction'], results['original'])

826


array([[825, 423],
       [  1,   1]], dtype=int64)

Again, unfortunately only 1 game is predicted to be worth watching! One way around this is to perhaps just watch games that the logistic regression predicts to be exciting with probability above a certain threshold. Or look at the probabilities of all the games being exciting to watch each weekend and watch the most likely.

In [18]:
alt_results = pd.DataFrame(data={'probs': probs, 'original': test['watch']})
alt_results['preds'] = alt_results['probs'].apply(lambda x: 1 if x > 0.35 else 0)
print(confusion_matrix(alt_results['preds'], alt_results['original']))
print((alt_results['original'] == alt_results['preds']).sum())

[[615 285]
 [211 139]]
754


Of course, more games are predicted to be worth watching if we lower thethreshold probability but we have introduced many false positives into the predictions.

As outlined in my blogpost, it seems that these predictors are exhibited evidence of collinearity. Therefore, it may be beneficial to drop some of the attributes and train again. I kept just HWW and AWW and retrained below.

In [19]:
train_ww = train[['HWW', 'AWW', 'watch']]
test_ww = test[['HWW', 'AWW', 'watch']]
clf_ww = LogisticRegression(solver='saga',max_iter=10000000, 
                            fit_intercept=True,
                           random_state=101)
clf_ww.fit(train_ww.drop('watch', axis=1), train_ww['watch'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=101, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
probs_ww = clf_ww.predict_proba(test_ww.drop('watch', axis=1))
probs_ww = [prob[1] for prob in probs_ww]

In [21]:
print(max(probs_ww))
print(min(probs_ww))

0.4730943548442019
0.2745230581583436


In [22]:
alt_results_ww = pd.DataFrame(data={'probs': probs_ww, 'original': test_ww['watch']})
alt_results_ww['preds'] = alt_results['probs'].apply(lambda x: 1 if x > 0.35 else 0)
print(confusion_matrix(alt_results_ww['preds'],alt_results_ww['original']))

[[615 285]
 [211 139]]


We see that still we are not getting very good results. We are predicting with about 60.5% accuracy and when we predict a game will be worth watching there is a 32.8% chance that it will actually be. Again this is only slightly better than random guessing as about 30% of games are worth watching. So a different approach is needed.